In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
drive.flush_and_unmount

<function google.colab.drive.flush_and_unmount(timeout_ms=86400000)>

In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.6 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-5sci0_bo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-5sci0_bo
  Resolved https://github.com/huggingface/accelerate to commit eba6eb79dc2ab652cd8b44b37165a4852768a8ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=226907 sha256=0994270baf9ac26af47d4f3c8bb9a388c468f9bf095fad5fd9dd1aa52f195e49
  Stored in directory: /tmp/pip-ephem-wheel-cache-a13w81mc/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
from os import listdir

import pandas as pd
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer, GPTNeoForCausalLM, GPT2TokenizerFast
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
LYRICS_GENRE = 'Jazz'

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP Finetuning/Genre Datasets/en_"+LYRICS_GENRE+"_Lyrics.csv")

print(data.shape)
data.head()

(1728, 3)


,Unnamed: 0,Lyric,Genres
0,18935,[Lil Wayne:]\nYoung Mulababy\n\nI Sit Here Lon...,Jazz
1,18938,"[Intro]\nAct a ass wit it, back ya ass wit it\...",Jazz
2,18941,Go!\nI'm in love with a beauty\nShe is a cutie...,Jazz
3,18944,"[Hook: Snl]\nI'm a thick motherfucker, I sit t...",Jazz
4,18947,"I'm bout to shoot at a nigga, no layup\nI pull...",Jazz


In [ ]:
data['length'] = data['Lyric'].apply(lambda x:len(str(x).split()))
data = data[data['length'] <= 300]

data = data.drop(['Unnamed: 0', 'Genres','length'], axis=1)

In [ ]:
data = data.sample(n=2000, random_state=42)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/generic.py:5773 in sample                    │
│                                                                                                  │
│    5770 │   │   if weights is not None:                                                          │
│    5771 │   │   │   weights = sample.preprocess_weights(self, weights, axis)                     │
│    5772 │   │                                                                                    │
│ ❱  5773 │   │   sampled_indices = sample.sample(obj_len, size, replace, weights, rs)             │
│    5774 │   │   result = self.take(sampled_indices, axis=axis)                                   │
│    5775 │   │                                                                                    │
│    5776 │   │   if ignore_index:                                                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/sample.py:150 in sample                      │
│                                                                                                  │
│   147 │   │   else:                                                                              │
│   148 │   │   │   raise ValueError("Invalid weights: weights sum to zero")                       │
│   149 │                                                                                          │
│ ❱ 150 │   return random_state.choice(obj_len, size=size, replace=replace, p=weights).astype(     │
│   151 │   │   np.intp, copy=False                                                                │
│   152 │   )                                                                                      │
│   153                                                                                            │
│                                                                                                  │
│ in numpy.random.mtrand.RandomState.choice:965                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
data.shape

(897, 1)

In [ ]:
data['Lyric'] = data['Lyric'].astype(str)

In [ ]:
data['Lyric'] = data['Lyric'].replace(r"[^\w\s']", '', regex=True)

In [ ]:
torch.cuda.empty_cache()

tokenizer = GPT2TokenizerFast.from_pretrained('EleutherAI/gpt-neo-125m', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m')
model.resize_token_embeddings(len(tokenizer))

torch.cuda.empty_cache()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')

else:
  device = torch.device('cpu')

print(device)

cpu


In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
train, test = train_test_split(data['Lyric'], test_size=0.15, shuffle=True, random_state=SEED)

In [ ]:
class ShapingDataset(Dataset):

    def __init__(self, texts):
        self.texts = texts
        self.tokenizer = tokenizer
        self.labels = []
        self.input_ids = []
        self.attention_masks = []
        self.max_length = max([len(re.findall('\w+', x)) for x in data['Lyric']]) # Lyric with most words.
        # self.max_length = maxword
        for text in texts:
            dici = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
            self.input_ids.append(dici['input_ids'][0])
            self.attention_masks.append(dici['attention_mask'][0])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [ ]:
train = ShapingDataset(train)
test = ShapingDataset(test)

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/outputs/',
    logging_dir = '/content/logs/',
    num_train_epochs = 20,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    logging_steps = 10000,
    save_steps = 10000,
    learning_rate = float('2e-4'),
    warmup_steps = 20)

In [ ]:
model.to(device)

trainer = Trainer(model=model, args=training_args,  train_dataset=train, eval_dataset=test, data_collator = lambda data: {
    'input_ids': torch.stack([f[0] for f in data]),
    'attention_mask': torch.stack([f[1] for f in data]),
    'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:
saved_root = "/content/drive/MyDrive/NLP Finetuning/Finetuned Models/"+LYRICS_GENRE+" Lyrics Generator"

In [ ]:
trainer.save_model(saved_root)

In [ ]:
model = GPTNeoForCausalLM.from_pretrained(saved_root)

In [ ]:
model.to('cuda')
generated = tokenizer(' ', return_tensors = 'pt').input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=10, max_length=100, top_p=0.95, temperature=2., num_return_sequences=5)

for i, sample_output in enumerate(sample_outputs):
  text = tokenizer.decode(sample_output.tolist())
  regex_padding = re.sub('<|pad|>', '', text)
  regex_barra = re.sub('[|+]', '', regex_padding)
  space = re.sub('[ +]', ' ', regex_barra)
  result = re.sub('[\n](2, )', '\n', space)
  print("\n==== Generated lyric {}:\n\n{}".format(i+1, result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



==== Generated lyric 1:

  my name is kimmy lie it is kimmy dana
Bing wants aotten bae i do it on kimmy i do it on kimmy

See bae i do it on a dream
Wake up and see bae
I do it on a dream
Well be gone

Cant take it
Cant take it
Cant take it
Could it be 
My heart cant take it
My heart cant take it

==== Generated lyric 2:

 

Sometimes when I wake up in the morning air
I see these pretty girls
And when I try to think
About their last thing they leave
So I bend down and pray
Im Lord knows I canDo it hard
Sometimes when I start to think
About their last thing they leave
So I bend down and pray
Im Lord knows I canDo it hard
Sometimes when I start to think
To bend down and pray
Im Lord knows I canDo it hard

==== Generated lyric 3:

  I wanna be with you girl
Oh yeah I wanna be with you
Real time pretty baby
Dont wanna be alone

Oh oh oh
Oh oh oh
I wanna be with you girl
Oh oh oh
I wanna be with you girl
Oh oh oh

Oh oh oh
Oh oh oh
Oh oh oh
I wanna be with you girl
Oh oh oh
I wanna be with